In [1]:
import yaml

In [2]:
with open("./dev/extra_config/nodes.yaml", "r") as file:
  node_info = yaml.safe_load(file)


In [3]:
node_info['nodes']

[{'node': 'thor',
  'gpu_type': '3090',
  'num_gpus': 2,
  'cpu_threads': 128,
  'gb_memory': 256},
 {'node': 'rym',
  'gpu_type': '3090',
  'num_gpus': 2,
  'cpu_threads': 16,
  'gb_memory': 16},
 {'node': 'atli',
  'gpu_type': 'L40S',
  'num_gpus': 2,
  'cpu_threads': 16,
  'gb_memory': 16}]

In [4]:
CONFIG = {
  "images": {
      "datascience": "brineylab/jupyterhub-datascience:v2024-08-13",
      "deeplearning": "brineylab/jupyterhub-deeplearning:v2024-08-13",
  },
  "gpu_counts": [1, 2, 4, 8],
  "node_info": node_info['nodes']
}

In [5]:
def _define_images(CONFIG):
      """Define the images for users, based on CONFIG"""
      return {
        "Image": {
          "display_name": "Image",
          "choices": {
              name: {
                  "display_name": name,
                  "default": (name == "datascience"),
                  "kubespawner_override": {"image": url},
              } for name, url in CONFIG["images"].items()
          },
          "unlisted_choice": {
            "enabled": True,
            "display_name": "Enter custom image",
            "display_name_in_choices": "other - select to provide a custom image",
            "validation_regex": '^.+/.+:.+$',
            "validation_message": 'Must be an image matching <user>/<name>:<tag>',
            "kubespawner_override": {
              "image": "{value}",
              "start_timeout": 600, # increase timeout to allow new images to be pulled
            },
          },
        }
      }

In [6]:
res = {
  "display_name": "CPU Server",
  "description": "Choose your image. <br> Datascience: default data analysis environment, including interpreters for Python and R. <br> Deeplearning: extends the data science environment to include ML libraries (such as torch, deepspeed, and 🤗).",
  "profile_options": {
    **_define_images(CONFIG),
  },
  "kubespawner_override": {
    "node_selector": {"node_profile": "cpu"},
  }
}


In [7]:
res

{'display_name': 'CPU Server',
 'description': 'Choose your image. <br> Datascience: default data analysis environment, including interpreters for Python and R. <br> Deeplearning: extends the data science environment to include ML libraries (such as torch, deepspeed, and 🤗).',
 'profile_options': {'Image': {'display_name': 'Image',
   'choices': {'datascience': {'display_name': 'datascience',
     'default': True,
     'kubespawner_override': {'image': 'brineylab/jupyterhub-datascience:v2024-08-13'}},
    'deeplearning': {'display_name': 'deeplearning',
     'default': False,
     'kubespawner_override': {'image': 'brineylab/jupyterhub-deeplearning:v2024-08-13'}}},
   'unlisted_choice': {'enabled': True,
    'display_name': 'Enter custom image',
    'display_name_in_choices': 'other - select to provide a custom image',
    'validation_regex': '^.+/.+:.+$',
    'validation_message': 'Must be an image matching <user>/<name>:<tag>',
    'kubespawner_override': {'image': '{value}', 'start_

In [8]:
def _define_num_gpus(CONFIG):
  """Define number of gpus for users, based on CONFIG"""
  return {
    "GPUNum": {
      "display_name": "Number of GPUs",
      "choices": {
        str(i): {
          "display_name": str(i),
          # config required to enable gpu: https://discourse.jupyter.org/t/set-runtimeclassname-for-gpu-enabled-images/21617/5
          "kubespawner_override": {
            "extra_resource_limits": {"nvidia.com/gpu": i},
            "extra_pod_config": {"runtimeClassName": "nvidia"},
          } if i > 0 else {},
        } for i in CONFIG["gpu_counts"]
      },
    }
  }
    
def _define_gpu_nodes(CONFIG):
  """Define node choices, based on CONFIG"""
  return {
    "Node": {
      "display_name": "Node",
      "choices": {
        str(node["node"]): { 
          "display_name": str(node["node"]),
          "kubespawner_override": {
              "node_selector": {"kubernetes.io/hostname": str(node["node"])},
          },
        }
        for node in CONFIG["node_info"] if "node" in node
      },
    }
  }

In [9]:
res = {
  "display_name": "GPU Server",
  "description": "Choose your image, node and number of GPUs.",
  "profile_options": {
    **_define_images(CONFIG),
    **_define_gpu_nodes(CONFIG),
    **_define_num_gpus(CONFIG),
  },
}

In [10]:
res

{'display_name': 'GPU Server',
 'description': 'Choose your image, node and number of GPUs.',
 'profile_options': {'Image': {'display_name': 'Image',
   'choices': {'datascience': {'display_name': 'datascience',
     'default': True,
     'kubespawner_override': {'image': 'brineylab/jupyterhub-datascience:v2024-08-13'}},
    'deeplearning': {'display_name': 'deeplearning',
     'default': False,
     'kubespawner_override': {'image': 'brineylab/jupyterhub-deeplearning:v2024-08-13'}}},
   'unlisted_choice': {'enabled': True,
    'display_name': 'Enter custom image',
    'display_name_in_choices': 'other - select to provide a custom image',
    'validation_regex': '^.+/.+:.+$',
    'validation_message': 'Must be an image matching <user>/<name>:<tag>',
    'kubespawner_override': {'image': '{value}', 'start_timeout': 600}}},
  'Node': {'display_name': 'Node',
   'choices': {'thor': {'display_name': 'thor',
     'kubespawner_override': {'node_selector': {'kubernetes.io/hostname': 'thor'}}}